In [0]:
!wget --no-check-certificate \
    https://storage.googleapis.com/laurencemoroney-blog.appspot.com/horse-or-human.zip \
    -O /tmp/horse-or-human.zip

--2020-02-13 12:46:54--  https://storage.googleapis.com/laurencemoroney-blog.appspot.com/horse-or-human.zip
Resolving storage.googleapis.com (storage.googleapis.com)... 172.217.194.128, 2404:6800:4003:c02::80
Connecting to storage.googleapis.com (storage.googleapis.com)|172.217.194.128|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 149574867 (143M) [application/zip]
Saving to: ‘/tmp/horse-or-human.zip’

/tmp/horse-or-human 100%[===================>] 142.65M   191MB/s    in 0.7s    

2020-02-13 12:46:55 (191 MB/s) - ‘/tmp/horse-or-human.zip’ saved [149574867/149574867]



In [0]:
import os
import zipfile
file_path='/tmp/horse-or-human.zip'
zip_ref=zipfile.ZipFile(file_path,'r')
zip_ref.extractall('/tmp/horse-or-human')
zip_ref.close()

In [0]:
train_horse_dir=os.path.join('/tmp/horse-or-human/horses')  #copy path of sub directory to the name
train_human_dir=os.path.join('/tmp/horse-or-human/humans')

In [0]:
train_horse_name=os.listdir(train_horse_dir)
print(train_horse_name[:10]) #prints the first ten names
train_human_name=os.listdir(train_human_dir)
print(train_human_name[:10])

print("\nNo of Horse Images is",len(train_horse_name))
print("\nNo of Human Images is",len(train_human_name))

['horse37-3.png', 'horse29-5.png', 'horse40-0.png', 'horse47-2.png', 'horse08-2.png', 'horse35-5.png', 'horse10-4.png', 'horse42-4.png', 'horse12-9.png', 'horse24-1.png']
['human01-04.png', 'human10-21.png', 'human06-24.png', 'human03-25.png', 'human17-03.png', 'human03-27.png', 'human05-22.png', 'human02-13.png', 'human06-05.png', 'human09-08.png']

No of Horse Images is 500

No of Human Images is 527


In [0]:
%matplotlib inline
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
nrows=4
ncols=4
fig=plt.gcf()
fig.set_size_inches(nrows*4,ncols*4)

horse_path=[os.path.join(train_horse_dir,train_horse_name[i])
      for i in range(0,8)]
human_path=[os.path.join(train_human_dir,train_human_name[i])
    for i in range(0,8)]
combined=horse_path+human_path
for i,toplot in enumerate(combined):
  sp=plt.subplot(nrows,ncols,i+1)
  sp.axis('off')
  img=mpimg.imread(toplot)
  plt.imshow(img)


In [0]:
import tensorflow as tf

In [0]:
model=tf.keras.models.Sequential([tf.keras.layers.Conv2D(64,(3,3),activation='relu',input_shape=(300,300,3)),
                                  tf.keras.layers.MaxPool2D((2,2)),
                                  #2nd Convolution
                                  tf.keras.layers.Conv2D(64,(3,3),activation='relu'),
                                  tf.keras.layers.MaxPool2D((2,2)),
                                  #3rd Convolution
                                  tf.keras.layers.Conv2D(64,(3,3),activation='relu'),
                                  tf.keras.layers.MaxPool2D((2,2)),
                                  #4th Convolution
                                  tf.keras.layers.Conv2D(64,(3,3),activation='relu'),
                                  tf.keras.layers.MaxPool2D((2,2)),
                                  #5th convolution
                                  tf.keras.layers.Conv2D(64,(3,3),activation='relu'),
                                  tf.keras.layers.MaxPool2D((2,2)),
                                  #flattening the image
                                  tf.keras.layers.Flatten(),
                                  #512 Neurons hidden layer
                                  tf.keras.layers.Dense(512,activation='relu'),
                                  #Final layer
                                  tf.keras.layers.Dense(1,activation='sigmoid')
                              ])

In [0]:
model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_5 (Conv2D)            (None, 298, 298, 64)      1792      
_________________________________________________________________
max_pooling2d_5 (MaxPooling2 (None, 149, 149, 64)      0         
_________________________________________________________________
conv2d_6 (Conv2D)            (None, 147, 147, 64)      36928     
_________________________________________________________________
max_pooling2d_6 (MaxPooling2 (None, 73, 73, 64)        0         
_________________________________________________________________
conv2d_7 (Conv2D)            (None, 71, 71, 64)        36928     
_________________________________________________________________
max_pooling2d_7 (MaxPooling2 (None, 35, 35, 64)        0         
_________________________________________________________________
conv2d_8 (Conv2D)            (None, 33, 33, 64)       

In [0]:
from tensorflow.keras.optimizers import RMSprop
model.compile(loss='binary_crossentropy',optimizer=RMSprop(lr=0.001),metrics=['acc'])

In [0]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator
train_datagen=ImageDataGenerator(rescale=1/255)
train_generator=train_datagen.flow_from_directory(
    '/tmp/horse-or-human',
    target_size=(300,300),
    batch_size=128,
    class_mode='binary'
)

Found 1027 images belonging to 2 classes.


In [0]:
history=model.fit_generator(train_generator,steps_per_epoch=8,epochs=15,verbose=1)

Epoch 1/15
8/8 [==============================] - 10s 1s/step - loss: 0.9212 - acc: 0.4980
Epoch 2/15
8/8 [==============================] - 6s 754ms/step - loss: 0.7008 - acc: 0.5413
Epoch 3/15
8/8 [==============================] - 7s 832ms/step - loss: 0.7347 - acc: 0.6908
Epoch 4/15
8/8 [==============================] - 7s 829ms/step - loss: 1.8544 - acc: 0.7152
Epoch 5/15
8/8 [==============================] - 8s 939ms/step - loss: 0.4962 - acc: 0.8750
Epoch 6/15
8/8 [==============================] - 7s 826ms/step - loss: 1.0038 - acc: 0.8865
Epoch 7/15
8/8 [==============================] - 7s 828ms/step - loss: 0.2683 - acc: 0.8765
Epoch 8/15
8/8 [==============================] - 6s 714ms/step - loss: 0.4083 - acc: 0.8953
Epoch 9/15
8/8 [==============================] - 8s 971ms/step - loss: 0.3412 - acc: 0.8770
Epoch 10/15
8/8 [==============================] - 7s 857ms/step - loss: 0.1097 - acc: 0.9522
Epoch 11/15
8/8 [==============================] - 7s 858ms/step - loss

In [0]:
from google.colab import files
import numpy as np
from keras.preprocessing import image